# Forest Cover Type Prediction - Logistic Regression


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

sns.set_theme(style="whitegrid")
warnings.filterwarnings('ignore')

print("Libraries loaded successfully.")

Libraries loaded successfully.


## Loading and Preprocessing Data

In [ ]:
df = pd.read_csv('covtype.csv')
print(f"Dataset shape: {df.shape}")

if df.duplicated().sum() > 0:
    df = df.drop_duplicates()

target_col = "Cover_Type"
numerical_cols = df.columns[:10]      
categorical_cols = df.columns[10:-1]  

X = df.drop(columns=[target_col])
y = df[target_col]

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = RobustScaler()

X_train_scaled = X_train.copy()
X_val_scaled = X_val.copy()

X_train_scaled[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_val_scaled[numerical_cols] = scaler.transform(X_val[numerical_cols])

print("Preprocessing complete.")

Dataset shape: (581012, 55)
Preprocessing complete.


## Logistic Regression Model implementations

In [5]:
lr_model = LogisticRegression(
    random_state=42,
)

lr_model.fit(X_train_scaled, y_train)

y_pred = lr_model.predict(X_val_scaled)

accuracy = accuracy_score(y_val, y_pred)
print(f"Validation Accuracy: {accuracy:.5f}\n")

print("Classification Report:")
print(classification_report(y_val, y_pred))

Validation Accuracy: 0.72355

Classification Report:
              precision    recall  f1-score   support

           1       0.71      0.70      0.70     42368
           2       0.75      0.80      0.77     56661
           3       0.68      0.81      0.73      7151
           4       0.59      0.43      0.50       549
           5       0.15      0.01      0.01      1899
           6       0.49      0.26      0.34      3473
           7       0.74      0.55      0.63      4102

    accuracy                           0.72    116203
   macro avg       0.59      0.51      0.53    116203
weighted avg       0.71      0.72      0.71    116203



adding some more parameters for tuning, lbfgs and saga were the most prominient solvers, but saga took a lot of time and was inefficient so i went forward with lbfgs only, kept 100 iterations only to keep things stable and working.

In [11]:
lr_model = LogisticRegression(
    random_state=42,
    solver='lbfgs',
    max_iter=100,
    multi_class='multinomial'
)

lr_model.fit(X_train_scaled, y_train)

y_pred = lr_model.predict(X_val_scaled)

accuracy = accuracy_score(y_val, y_pred)
print(f"Validation Accuracy: {accuracy:.5f}\n")

print("Classification Report:")
print(classification_report(y_val, y_pred))

Validation Accuracy: 0.72355

Classification Report:
              precision    recall  f1-score   support

           1       0.71      0.70      0.70     42368
           2       0.75      0.80      0.77     56661
           3       0.68      0.81      0.73      7151
           4       0.59      0.43      0.50       549
           5       0.15      0.01      0.01      1899
           6       0.49      0.26      0.34      3473
           7       0.74      0.55      0.63      4102

    accuracy                           0.72    116203
   macro avg       0.59      0.51      0.53    116203
weighted avg       0.71      0.72      0.71    116203



More parameters added for tuning

In [14]:
lr_model = LogisticRegression(
    random_state=42,
    solver='lbfgs',
    max_iter=100,
    multi_class='multinomial',
    C=1.0,                     
    tol=1e-5,                  
    penalty='l2',              
    class_weight=None       
)

lr_model.fit(X_train_scaled, y_train)

y_pred = lr_model.predict(X_val_scaled)

accuracy = accuracy_score(y_val, y_pred)
print(f"Validation Accuracy: {accuracy:.5f}\n")

print("Classification Report:")
print(classification_report(y_val, y_pred))


Validation Accuracy: 0.72355

Classification Report:
              precision    recall  f1-score   support

           1       0.71      0.70      0.70     42368
           2       0.75      0.80      0.77     56661
           3       0.68      0.81      0.73      7151
           4       0.59      0.43      0.50       549
           5       0.15      0.01      0.01      1899
           6       0.49      0.26      0.34      3473
           7       0.74      0.55      0.63      4102

    accuracy                           0.72    116203
   macro avg       0.59      0.51      0.53    116203
weighted avg       0.71      0.72      0.71    116203



Based on some manual tunings and some observations from the above models, i tried to do an optuna searhc on wide ranges in order to find the best hyperparameters.
Since the data is too large (~ 600000 rows), training on whole dataset can give diminishing results, so i used stratified sub sampling, so that we coul train it accurately.

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
import optuna
from optuna.samplers import TPESampler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

df_sub, _ = train_test_split(
    df,
    train_size=150000,
    random_state=42,
    stratify=df['Cover_Type']
)

X = df_sub.drop(columns=['Cover_Type'])
y = df_sub['Cover_Type']

X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

def objective(trial):
    C = trial.suggest_loguniform('C', 1e-3, 1e2)
    tol = trial.suggest_loguniform('tol', 1e-6, 1e-3)
    max_iter = trial.suggest_int('max_iter', 200, 1200)
    class_weight = trial.suggest_categorical('class_weight', [None, 'balanced'])

    model = LogisticRegression(
        random_state=42,
        solver='lbfgs',
        penalty='l2',
        C=C,
        tol=tol,
        max_iter=max_iter,
        class_weight=class_weight
    )

    score = cross_val_score(
        model,
        X_train_scaled,
        y_train,
        cv=3,
        scoring='accuracy',
        n_jobs=-1
    ).mean()

    return score

study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42))
study.optimize(objective, n_trials=50, show_progress_bar=True)

print("Best Trial:")
best_params = study.best_trial.params
print(best_params)
print(f"Best CV Accuracy: {study.best_trial.value:.5f}")


[I 2025-12-10 12:18:43,162] A new study created in memory with name: no-name-38486456-23f9-4e11-a7d1-21ecee956c94


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-12-10 12:18:46,486] Trial 0 finished with value: 0.7207166666666667 and parameters: {'C': 0.0745934328572655, 'tol': 0.0007114476009343421, 'max_iter': 932, 'class_weight': None}. Best is trial 0 with value: 0.7207166666666667.
[I 2025-12-10 12:18:51,594] Trial 1 finished with value: 0.5740833333333334 and parameters: {'C': 0.0060252157362038605, 'tol': 1.493656855461763e-06, 'max_iter': 1067, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.7207166666666667.
[I 2025-12-10 12:18:53,366] Trial 2 finished with value: 0.6917416666666666 and parameters: {'C': 0.001267425589893723, 'tol': 0.0008123245085588687, 'max_iter': 1033, 'class_weight': None}. Best is trial 0 with value: 0.7207166666666667.
[I 2025-12-10 12:18:57,531] Trial 3 finished with value: 0.7139833333333333 and parameters: {'C': 0.008260808399079604, 'tol': 8.179499475211674e-06, 'max_iter': 725, 'class_weight': None}. Best is trial 0 with value: 0.7207166666666667.


/home/iiitb/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/iiitb/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regr

[I 2025-12-10 12:19:14,511] Trial 4 finished with value: 0.595425 and parameters: {'C': 1.1462107403425035, 'tol': 2.62108787826544e-06, 'max_iter': 492, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.7207166666666667.


...

[I 2025-12-10 12:29:57,424] Trial 47 finished with value: 0.7230416666666667 and parameters: {'C': 22.43374811925354, 'tol': 2.538463188780518e-06, 'max_iter': 717, 'class_weight': None}. Best is trial 24 with value: 0.7231333333333333.


/home/iiitb/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/iiitb/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regr

[I 2025-12-10 12:30:07,471] Trial 48 finished with value: 0.723075 and parameters: {'C': 6.130972124714427, 'tol': 9.9644139621569e-06, 'max_iter': 325, 'class_weight': None}. Best is trial 24 with value: 0.7231333333333333.


/home/iiitb/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/iiitb/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regr

[I 2025-12-10 12:30:23,562] Trial 49 finished with value: 0.7230166666666666 and parameters: {'C': 1.9009107183117988, 'tol': 1.0038914233752853e-06, 'max_iter': 475, 'class_weight': None}. Best is trial 24 with value: 0.7231333333333333.
Best Trial:
{'C': 26.88601840780637, 'tol': 2.4326257523211373e-06, 'max_iter': 562, 'class_weight': None}
Best CV Accuracy: 0.72313


In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

best_lr = LogisticRegression(
    random_state=42,
    solver='lbfgs',
    penalty='l2',
    C=best_params['C'],
    tol=best_params['tol'],
    max_iter=best_params['max_iter'],
    class_weight=best_params['class_weight']
)

best_lr.fit(X_train_scaled, y_train)

y_pred = best_lr.predict(X_val_scaled)

print(f"Validation Accuracy: {accuracy_score(y_val, y_pred):.5f}")
print(classification_report(y_val, y_pred))


Validation Accuracy: 0.72357
              precision    recall  f1-score   support

           1       0.71      0.69      0.70     10938
           2       0.75      0.80      0.77     14628
           3       0.68      0.80      0.73      1846
           4       0.62      0.46      0.53       142
           5       0.22      0.00      0.01       490
           6       0.48      0.25      0.33       897
           7       0.72      0.54      0.62      1059

    accuracy                           0.72     30000
   macro avg       0.60      0.51      0.53     30000
weighted avg       0.71      0.72      0.71     30000



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
import optuna
from optuna.samplers import TPESampler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore")

df_sub, _ = train_test_split(
    df,
    train_size=300000,
    random_state=42,
    stratify=df['Cover_Type']
)

X = df_sub.drop(columns=['Cover_Type'])
y = df_sub['Cover_Type']

X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

def objective(trial):
    C = trial.suggest_loguniform('C', 1e-3, 1e2)
    tol = trial.suggest_loguniform('tol', 1e-6, 1e-3)
    max_iter = trial.suggest_int('max_iter', 500, 2000)
    class_weight = trial.suggest_categorical('class_weight', [None])

    model = LogisticRegression(
        random_state=42,
        solver='lbfgs',
        penalty='l2',
        C=C,
        tol=tol,
        max_iter=max_iter,
        class_weight=class_weight
    )

    score = cross_val_score(
        model,
        X_train_scaled,
        y_train,
        cv=4,
        scoring='accuracy',
        n_jobs=-1
    ).mean()

    return score

study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42))
study.optimize(objective, n_trials=100, show_progress_bar=True)

print("Best Trial:")
best_params = study.best_trial.params
print(best_params)
print(f"Best CV Accuracy: {study.best_trial.value:.5f}")


[I 2025-12-10 12:39:20,211] A new study created in memory with name: no-name-5d9eada3-46ff-4487-9151-c945b0544c63


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2025-12-10 12:39:27,469] Trial 0 finished with value: 0.7222875 and parameters: {'C': 0.0745934328572655, 'tol': 0.0007114476009343421, 'max_iter': 1598, 'class_weight': None}. Best is trial 0 with value: 0.7222875.


/home/iiitb/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/iiitb/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regr

[I 2025-12-10 12:40:33,450] Trial 1 finished with value: 0.7241875 and parameters: {'C': 0.9846738873614566, 'tol': 2.9380279387035354e-06, 'max_iter': 734, 'class_weight': None}. Best is trial 1 with value: 0.7241875.
[I 2025-12-10 12:40:36,647] Trial 2 finished with value: 0.7090916666666667 and parameters: {'C': 0.0019517224641449498, 'tol': 0.0003967605077052988, 'max_iter': 1402, 'class_weight': None}. Best is trial 1 with value: 0.7241875.
[I 2025-12-10 12:42:09,549] Trial 3 finished with value: 0.7241958333333334 and parameters: {'C': 3.4702669886504163, 'tol': 1.1527987128232402e-06, 'max_iter': 1955, 'class_weight': None}. Best is trial 3 with value: 0.7241958333333334.


...

/home/iiitb/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/iiitb/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regr

[I 2025-12-10 14:32:34,791] Trial 98 finished with value: 0.7243333333333333 and parameters: {'C': 73.56863800136883, 'tol': 4.344210071456278e-06, 'max_iter': 876, 'class_weight': None}. Best is trial 54 with value: 0.7243375.


/home/iiitb/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/iiitb/anaconda3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regr

[I 2025-12-10 14:34:50,734] Trial 99 finished with value: 0.7242958333333334 and parameters: {'C': 30.08999565305235, 'tol': 4.498810688797412e-06, 'max_iter': 836, 'class_weight': None}. Best is trial 54 with value: 0.7243375.
Best Trial:
{'C': 66.41584069964382, 'tol': 5.470549980644126e-05, 'max_iter': 1478, 'class_weight': None}
Best CV Accuracy: 0.72434


Exception ignored in: <function ResourceTracker.__del__ at 0x7367a4d7e020>
Traceback (most recent call last):
  File "/home/iiitb/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/home/iiitb/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/home/iiitb/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x7f4af6992020>
Traceback (most recent call last):
  File "/home/iiitb/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/home/iiitb/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/home/iiitb/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ 

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

best_lr = LogisticRegression(
    random_state=42,
    solver='lbfgs',
    penalty='l2',
    C=best_params['C'],
    tol=best_params['tol'],
    max_iter=best_params['max_iter'],
    class_weight=best_params['class_weight']
)

best_lr.fit(X_train_scaled, y_train)

y_pred = best_lr.predict(X_val_scaled)

print(f"Validation Accuracy: {accuracy_score(y_val, y_pred):.5f}")
print(classification_report(y_val, y_pred))


Validation Accuracy: 0.72480
              precision    recall  f1-score   support

           1       0.71      0.70      0.71     21876
           2       0.75      0.80      0.77     29256
           3       0.68      0.80      0.73      3692
           4       0.63      0.48      0.54       284
           5       0.39      0.01      0.02       981
           6       0.48      0.28      0.35      1793
           7       0.73      0.57      0.64      2118

    accuracy                           0.72     60000
   macro avg       0.62      0.52      0.54     60000
weighted avg       0.72      0.72      0.72     60000

